### se importan las librerias necesarias

In [75]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

# se cargan las hojas de excel

In [76]:
totales_accesos_tecnologia = pd.read_excel('../data/Internet.xlsx', sheet_name='Totales Accesos Por Tecnología')
totales_accesos_tecnologia

,Año,Trimestre,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total,Periodo
0,2023,4,836390,6022532,3908183,585760,194796,11547661,Oct-Dic 2023
1,2023,3,897895,6018832,3708718,581436,200027,11406908,Jul-Sept 2023
2,2023,2,1006509,5997149,3463988,581823,202428,11251897,Abr-Jun 2023
3,2023,1,1204417,6002582,3243411,576647,214838,11241895,Ene-Mar 2023
4,2022,4,1315373,6037457,3060575,572969,221740,11208114,Oct-Dic 2022
5,2022,3,1395277,6031970,2871541,557110,235230,11091128,Jul-Sept 2022
6,2022,2,1468333,5979214,2723285,556243,231609,10958684,Abr-Jun 2022
7,2022,1,1533240,6073426,2219533,545814,251996,10624009,Ene-Mar 2022
8,2021,4,1657615,5984240,2072236,523107,252596,10489794,Oct-Dic 2021
9,2021,3,1950631,5826257,1566048,492415,250190,10085541,Jul-Sept 2021


In [77]:
accesos_tecnologia = pd.read_excel('../data/Internet.xlsx', sheet_name='Accesos Por Tecnología')
accesos_tecnologia

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
0,2023,4,Buenos Aires,223391.0,2797700.0,1784593.0,137993.0,67943.0,5011620.0
1,2023,4,Capital Federal,58700.0,1198835.0,182520.0,4492.0,29820.0,1474367.0
2,2023,4,Catamarca,6669.0,11016.0,55437.0,1384.0,81.0,74587.0
3,2023,4,Chaco,14000.0,60277.0,62301.0,8404.0,2404.0,147386.0
4,2023,4,Chubut,26306.0,77217.0,26174.0,30543.0,9930.0,170170.0
...,...,...,...,...,...,...,...,...,...
957,2014,1,Santiago Del Estero,32567.0,3598.0,19.0,915.0,34.0,37133.0
958,2014,1,Tierra Del Fuego,21618.0,2837.0,648.0,1.0,2934.0,28038.0
959,2014,1,Tucumán,129717.0,83.0,121.0,13.0,98.0,130032.0
960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
#Totales Accesos por velocidad
totales_accesos_velocidad = pd.read_excel('../data/Internet.xlsx', sheet_name='Totales Accesos por velocidad')
totales_accesos_velocidad

,Año,Trimestre,Hasta 512 kbps,Entre 512 Kbps y 1 Mbps,Entre 1 Mbps y 6 Mbps,Entre 6 Mbps y 10 Mbps,Entre 10 Mbps y 20 Mbps,Entre 20 Mbps y 30 Mbps,Más de 30 Mbps,OTROS,Total
0,2024,4,29708,71742,900253,978108,697232,350290,8224736,295592,11547661
1,2023,3,30274,80609,945608,998725,720084,350918,8003269,277421,11406908
2,2023,2,30827,79405,963510,1028250,734125,364730,7775095,275955,11251897
3,2023,1,31437,82048,1001553,1060321,729194,357769,7564819,414754,11241895
4,2022,4,31981,85668,1075272,1109421,765015,335573,7410902,394282,11208114
5,2022,3,33013,96727,1161370,1151906,823505,479822,6995750,349035,11091128
6,2022,2,33667,99498,1193090,1197030,856562,485321,6741922,351594,10958684
7,2022,1,34890,104840,1263273,1209148,967508,509830,6336187,198333,10624009
8,2021,4,41262,28521,1413208,1245333,976539,558358,6032322,194251,10489794
9,2021,3,40174,41437,2550229,1095772,710122,536364,4948174,163269,10085541


In [79]:
accesos_velocidad = pd.read_excel('../data/Internet.xlsx', sheet_name='Accesos por velocidad')
accesos_velocidad

,Año,Trimestre,Provincia,HASTA 512 kbps,+ 512 Kbps - 1 Mbps,+ 1 Mbps - 6 Mbps,+ 6 Mbps - 10 Mbps,+ 10 Mbps - 20 Mbps,+ 20 Mbps - 30 Mbps,+ 30 Mbps,OTROS,Total
0,2024,4,Buenos Aires,26909.0,23962.0,228393.0,239339.0,246144.0,110876.0,4042646.0,93351.0,5011620.0
1,2024,4,Capital Federal,516.0,4544.0,26409.0,48083.0,33925.0,5829.0,1355061.0,0.0,1474367.0
2,2024,4,Catamarca,71.0,209.0,3065.0,3734.0,4423.0,1827.0,61181.0,77.0,74587.0
3,2024,4,Chaco,236.0,477.0,14811.0,10361.0,6247.0,5729.0,105485.0,4040.0,147386.0
4,2024,4,Chubut,111.0,1088.0,44276.0,31402.0,22843.0,20196.0,33665.0,16589.0,170170.0
...,...,...,...,...,...,...,...,...,...,...,...,...
955,2014,1,Santa Cruz,161.0,1625.0,24972.0,1.0,1.0,0.0,0.0,0.0,26760.0
956,2014,1,Santa Fe,8456.0,124468.0,345225.0,20328.0,6845.0,23.0,668.0,0.0,506013.0
957,2014,1,Santiago Del Estero,1234.0,10531.0,22817.0,2422.0,109.0,0.0,0.0,0.0,37113.0
958,2014,1,Tierra Del Fuego,12.0,607.0,30902.0,6.0,0.0,0.0,0.0,0.0,31527.0


In [80]:
penetracion_totales = pd.read_excel('../data/Internet.xlsx', sheet_name='Penetracion-totales')
penetracion_totales

,Año,Trimestre,Accesos por cada 100 hogares,Accesos por cada 100 hab,Periodo
0,2023,4,78.560000,24.660000,Oct-Dic 2023
1,2023,3,77.840000,24.410000,Jul-Sept 2023
2,2023,2,77.020000,24.140000,Abr-Jun 2023
3,2023,1,77.200000,24.170000,Ene-Mar 2023
4,2022,4,77.210000,24.150000,Oct-Dic 2022
5,2022,3,76.640000,23.950000,Jul-Sept 2022
6,2022,2,75.965424,23.720442,Abr-Jun 2022
7,2022,1,73.878803,23.049171,Ene-Mar 2022
8,2021,4,73.176397,22.810702,Oct-Dic 2021
9,2021,3,70.578853,21.982550,Jul-Sept 2021


In [81]:
penetracion_poblacion = pd.read_excel('../data/Internet.xlsx', sheet_name='Penetración-poblacion')
penetracion_poblacion

,Año,Trimestre,Provincia,Accesos por cada 100 hab
0,2023,4,Buenos Aires,27.680000
1,2023,4,Capital Federal,47.800000
2,2023,4,Catamarca,17.460000
3,2023,4,Chaco,11.850000
4,2023,4,Chubut,26.040000
...,...,...,...,...
955,2014,1,Santa Cruz,8.677327
956,2014,1,Santa Fe,15.077801
957,2014,1,Santiago Del Estero,4.058899
958,2014,1,Tierra Del Fuego,19.104921


In [82]:
penetracion_hogares = pd.read_excel('../data/Internet.xlsx', sheet_name='Penetracion-hogares')
penetracion_hogares

,Año,Trimestre,Provincia,Accesos por cada 100 hogares
0,2023,4,Buenos Aires,80.960000
1,2023,4,Capital Federal,117.020000
2,2023,4,Catamarca,68.630000
3,2023,4,Chaco,44.350000
4,2023,4,Chubut,84.960000
...,...,...,...,...
955,2014,1,Santa Cruz,28.939093
956,2014,1,Santa Fe,47.847663
957,2014,1,Santiago Del Estero,16.309924
958,2014,1,Tierra Del Fuego,63.965926


# se realiza el ETL

In [83]:
totales_accesos_tecnologia.drop(columns= 'Periodo', inplace= True)
#la columna no es nesesaria ya que año y trimestre cumplen su misma funcion
accesos_tecnologia.drop([960, 961], inplace= True) # las columnas no tienen datos
accesos_tecnologia['Año'] = accesos_tecnologia['Año'].apply(lambda x:x[:4] if type(x) == str else x)
accesos_tecnologia['Trimestre'] = accesos_tecnologia['Trimestre'].apply(lambda x:x[:1] if type(x) == str else x)
#totales_accesos_velocidad no nesesita ninguna transformacion
accesos_velocidad = accesos_velocidad.fillna(0) #se rellenan los vacios con el valor 0
accesos_velocidad['OTROS'] = accesos_velocidad['OTROS'].apply(abs)
# hay un valor negativo lo cual es imposible en este contexto por lo que se usa la funcion de valor absoluto
#la columna no es nesesaria ya que año y trimestre cumplen su misma funcion
penetracion_totales = penetracion_totales.drop(columns='Periodo')
#penetracion_poblacion no nesesita ninguna transformacion
#penetracion_hogares no nesesita ninguna transformacion